In [1]:
#Importing required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from datetime import datetime
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from tqdm import tqdm_notebook as tqdmn

In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())

In [2]:
#Files on website are split into the varios months so looping through to read them all
df_list = []
for i in range(1, 13):
    if i <10:
         df_list.append(pd.read_csv(f"Bike share ridership 2021-0{i}.csv",encoding='cp1252'))
    else:
        df_list.append(pd.read_csv(f"Bike share ridership 2021-{i}.csv",encoding='cp1252'))
   


In [3]:
#concatenating all data
df = pd.concat(df_list)

In [4]:
#Error with Trip Id first fill missing values
df = df.rename(columns={'ï»¿Trip Id':'Trip ID 2'})
df['Trip Id'] = df['Trip Id'].fillna("NEW VALUE!")


In [1]:
Correcting Trip ID
def fix_trip(ID, ID2 ):
    if ID == "NEW VALUE!":
        return ID2
    else:
        return ID
        
df['Trip Id'] = df[['Trip Id','Trip ID 2']].apply(lambda df: fix_trip(df['Trip Id'],df['Trip ID 2']),axis=1)

In [ ]:
#Dropping duplicate error column
df = df.drop(['Trip ID 2'], axis=1)

In [ ]:
#Geographic data of stops seems interesting will be adding longtitude and lattitude to graph this info
my_list = []
for item in df['Start Station Name'].unique():
    my_list.append(item)

In [ ]:
#Simplifying dataframe for only necessary info
df3 = df[['Start Station Name','Trip Id']]
df3 = df3.set_index('Trip Id')

In [ ]:
#For webscraping purposes removing the / between street names to avoid interfering with URL
df3['Start Station Name']= df3['Start Station Name'].str.replace('/', 'and')

In [ ]:
#Constructing URL's that we will scrape info from
df3['Url'] = ['https://www.google.com/maps/search/' + i for i in df3['Start Station Name'] ]

In [ ]:
#Many duplicate URL's in set as the stations repeat, this will reduce to only unique urls/addresses
df3 = df3.drop_duplicates(subset=['Url'])

In [ ]:
#Scraping google maps using URL's to obtain URL's including the coordinates
Url_With_Coordinates = []

option = webdriver.ChromeOptions()
prefs = {'profile.default_content_setting_values': {'images':2, 'javascript':2}}
option.add_experimental_option('prefs', prefs)

driver = webdriver.Chrome(executable_path="/Users/seifwork/Desktop/chromedriver", options=option)

for url in tqdmn(df3['Url'], leave=False):
    driver.get(url)
    Url_With_Coordinates.append(driver.find_element_by_css_selector('meta[itemprop=image]').get_attribute('content'))
    
driver.close()

In [ ]:
#Adding URL's with coordinates to dataframe
df3 = df3.assign(Url_with_coordinates=Url_With_Coordinates)

In [ ]:
#Creating functions that will strip only the station coordinates  from the google url both latitude and longtitude
def lat(Url_with_coordinates):
    return Url_with_coordinates.split('?center=')[1].split('&zoom=')[0].split('%2C')[0]
def long(Url_with_coordinates):
    return Url_with_coordinates.split('?center=')[1].split('&zoom=')[0].split('%2C')[1]

In [ ]:
#Applying function to dataframe to create new columns with coordinates
df4['lat'] = df4['Url_with_coordinates'].apply(lat)
df4['long'] = df4['Url_with_coordinates'].apply(long)

In [ ]:
#Combining latitude and longtitude into one column
df3['lat and long'] = df3['lat'] +','+ df3['long']

In [ ]:
#Creating a dictionary connecting station names with the latitude and longtitude
d = dict(zip(df3['Start Station Name'], df3['lat and long']))

In [ ]:
#Reassinging lat and long values for each station name based on dictionary
df3["lat and long"] = df3["Start Station Name"].apply(lambda x: d.get(x))

In [ ]:
#renaming to differentiate between coordinates for starting station
df3 = df3.rename(columns={"lat and long": "lat and long start"})

In [ ]:
#Repeating for end station coordinates
df3["lat and long end"] = df3["End Station Name"].apply(lambda x: d.get(x))

In [ ]:
#Splitting columns into separate latitude and longtitude again
df["lat start"]= df["lat and long start"].str.split(",", n = 1, expand = False).str[0]
df["long start"]= df["lat and long start"].str.split(",", n = 1, expand = False).str[1]
df["lat end"]= df["lat and long end"].str.split(",", n = 1, expand = False).str[0]
df["long end"]= df["lat and long end"].str.split(",", n = 1, expand = False).str[1]

In [ ]:
#Dropping unnecessary columns 
df = df.drop(['lat and long start', 'lat and long end'], axis=1)

In [ ]:
#Exporting to csv to import into Tableau
df3.to_csv('combined_bikeshare-.csv',index=False)

**Now that the data has been cleaned and coordinate info added we can import this into Tableau for analysis and visualization**

